# 발표 소단원 리스트

12-2,12-6,12-10,

# 전체 요약자료

-


# 문항별 자료 
- (12-2) thread가 작동 중인지 알아보기

- (12-6) Thread specific(local)한 데이터 저장 방법

- (12-10) Actor task 정의하기 : concurrency


### (12-2) Determinig if a thread has started
- (sol) threading 라이브러리에 Event 객체 사용

In [1]:
## wait for something to happen!!

from threading import Thread,Event
import time

# code to execute in an independent thread
def countdown(n,started_evt):
    print('Countdown Starting')
    started_evt.set()
    while n > 0:
        print('T-minus',n)
        n -= 1
        time.sleep(5)
        
started_evt = Event()

print('Launching countdown')
t = Thread(target = countdown,args=(10,started_evt))
t.start()

started_evt.wait()
print('countdown is running')

Launching countdown
Countdown Starting
T-minuscountdown is running
 10
T-minus 9
T-minus 8
T-minus 7
T-minus 6
T-minus 5
T-minus 4
T-minus 3
T-minus 2
T-minus 1


In [ ]:
## event object suits for one-time event
## for multiple times -> using Condition object

In [3]:
import threading
import time

class PeriodicTimer:
    def __init__(self,interval):
        self._interval = interval
        self._flag = 0
        self._cv = threading.Condition()
        
    def start(self):
        t = threading.Thread(target = self.run)
        t.daemon = True
        t.start()
        
    def run(self):
        '''
        run the timer and notify waiting threads after each interval
        '''
        while True:
            time.sleep(self._interval)
            with self._cv:
                self._flag ^= 1
                self._cv.notify_all()
                
    def wait_for_tick(self):
        '''
        wait for the next tick of the timer
        '''
        with self._cv:
            last_flag = self._flag
            while last_flag == self._flag:
                self._cv.wait()
                
ptimer = PeriodicTimer(5)
ptimer.start()

## Two threads that synchronize on the timer

def countdown(nticks):
    while nticks > 0:
        ptimer.wait_for_tick()
        print('T-minus',nticks)
        nticks -= 1
        
def countup(last):
    n = 0
    while n < last:
        ptimer.wait_for_tick()
        print('Counting',n)
        n += 1
        
threading.Thread(target = countdown,args=(10,)).start()
threading.Thread(target = countup,args=(5,)).start()

T-minusCounting 0
 10
CountingT-minus 9
 1
T-minusCounting 2
 8
CountingT-minus 7
 3
T-minusCounting 4 6

T-minus 5
T-minus 4
T-minus 3
T-minus 2
T-minus 1


### (12-6) Storing Thread-Specific State
- (sol) threading.local() 사용

In [1]:
from socket import socket,AF_INET,SOCK_STREAM
import threading

class LazyConnection:
    def __init__(self,address,family = AF_INET, type_ = SOCK_STREAM):
        self.address = address
        self.family = family
        self.type = type_
        self.local = threading.local()
        
    def __enter__(self):
        if hasattr(self.local,'sock'):
            raise RuntimeError('Already Connected')
        self.local.sock = socket(self.family,self.type)
        self.local.sock.connect(self.address)
        return self.local.sock
    
    def __exit__(self,exc_ty,exc_val,tb):
        self.local.sock.close()
        del self.local.sock

In [3]:
from functools import partial
def test(conn):
    with conn as s:
        s.send(b'GET /index.html HTTP/1.0\r\n')
        s.send(b'Host: www.python.org\r\n')
        s.send(b'\r\n')
        resp = b''.join(iter(partial(s.recv,8192),b''))
        
    print('Got {} bytes \n'.format(len(resp)))
    
conn = LazyConnection(('www.python.org',80))
t1 = threading.Thread(target=test,args=(conn,))
t2 = threading.Thread(target=test,args=(conn,))
t1.start()
t2.start()
t1.join()
t2.join()

Got 392 bytes 

Got 392 bytes 



### (12-10) Defining an Actor Task
- (sol) "actor model" 

In [4]:
from queue import Queue
from threading import Thread,Event

class ActorExit(Exception):
    pass

class Actor:
    def __init__(self):
        self._mailbox = Queue()
        
    def send(self,msg):
        self._mailbox.put(msg)
        
    def recv(self):
        msg = self._mailbox.get()
        if msg is ActorExit:
            raise ActorExit()
        return msg
    
    def close(self):
        self.send(ActorExit)
        
    
    def start(self):
        self._terminated = Event()
        t = Thread(target = self._bootstrap)
        t.daemon = True
        t.start()
        
    def _bootstrap(self):
        try:
            self.run()
        except ActorExit:
            pass
        finally:
            self._terminated.set()
            
    def join(self):
        self._terminated.wait()
        
    def run(self):
        while True:
            msg = self.recv()
            
class PrintActor(Actor):
    def run(self):
        while True:
            msg = self.recv()
            print('Got:',msg)

In [2]:
p = PrintActor()
p.start()

In [3]:
p.send('Hello')
p.send('World')

Got: Hello
Got: World


In [4]:
p.close()

In [5]:
p.join()

In [1]:
### gneerator 사용 예제
def print_actor():
    while True:
        try:
            msg = yield
            print('Got:',msg)
        except GeneratorExit:
            print('Actor terminating')

In [2]:
p = print_actor()
next(p) # advance to the yield (ready to receive)
p.send("Hello")
p.send("World")
p.close()

Got: Hello
Got: World
Actor terminating


RuntimeError: generator ignored GeneratorExit

In [5]:
## pass tagged message
class TaggedActor(Actor):
    def run(self):
        while True:
            tag,*payload = self.recv()
            getattr(self,'do_'+tag)(*payload)
            
            
    def do_A(self,x):
        print("Running A",x)
        
    def do_B(self,x,y):
        print('Running B',x,y)
        
a = TaggedActor()
a.start()
a.send(('A',1))
a.send(('B',2,3))

Running A 1
Running B 2 3


In [6]:
a.close()

In [7]:
a.join()

In [8]:
### arbitrary functions to be executed! 
## and results to be communicated back using a special Result object
from threading import Event
class Result:
    def __init__(self):
        self._evt = Event()
        self._result = None
        
    def set_result(self,value):
        self._result = value
        self._evt.set()
        
    def result(self):
        self._evt.wait()
        return self._result
    
class Worker(Actor):
    def submit(self,func,*args,**kwargs):
        r = Result()
        self.send((func,args,kwargs,r))
        return r
    
    def run(self):
        while True:
            func,args,kwargs, r = self.recv()
            r.set_result(func(*args,**kwargs))
            
worker = Worker()
worker.start()
r = worker.submit(pow,2,3)
print(r.result())

8
